#### 1. Click on the Select Kernel icon in top right of this window and choose ```Python Environments...```
- The venv environment with the star next to it is the right choice
- This is the python venv where the client was just installed

#### 2. The code cell below will import the necessary ctm_python_client modules for our Workflow
- Hover your mouse to the left side of cell below to invoke the play button for that cell.
- Since this is the first time running code cells, VSCode will automatically install ipykenel as seen bottom right first.
- Upons successful completion of the code cell you wil have a green check mark in the bottom left of cell.

In [ ]:
from ctm_python_client.core.workflow import Workflow, WorkflowDefaults
from ctm_python_client.core.comm import Environment
from aapi.job import *

#### 3. The next code cell creates the "workflow" object.  Its sort of like a Workspace for adding folders and jobs.
- Here you are creating the workbench environment
- You may also setup any folder/job defaults like the run_as user

In [ ]:
workflow = Workflow(
    Environment.create_workbench(host='workbench'),
    WorkflowDefaults(
        run_as='workbench',     # defining workbench as the user to run the jobs by default
    )
)

#### 4. Next we will add 2 jobs the workflow
- Job1 and Job2 are the jobnames that will be added to the Second_Workflow folder
- Each is an OS Command job that runs the echo command.

In [ ]:
# the .add method returns the path of the job which you can use to connect it with other jobs
firstjob = workflow.add(
    JobCommand('Job1', command='echo "I run first"'),
    inpath='MyFolder'
)

secondjob = workflow.add(
    JobCommand('Job2', command='echo "I run second"'),
    inpath='Second_Workflow'
)

#### 5. We'll use the workflow connect method to create a dependency between jobs.

In [ ]:
print(f'Connecting {firstjob} to {secondjob}')
workflow.connect(firstjob, secondjob)

#### 6. We can visualize our workflow
- Graphviz has been installed in order to make the visualizaiton
- The ctm_python_client implements graphviz

In [ ]:
from ctm_python_client.ext.viz import get_graph

get_graph(workflow)

#### 7. Let's run it
- Feedback from the run is stored in the run object
- The workflow supports additional methods like ```build``` for validation and ```deploy``` for checking into Control-M

In [ ]:
run = workflow.run()

#### 8. We can use the run object to monitor all aspects of the executing workflow
- ```print_statuses``` shows current job and folder status
- ```print_output``` and ```get_output``` for a specific job
- Use ```get_status``` to retrieve job details
- Performa other operations on jobs like ```rerun```, ```hold```, and ```kill```

In [ ]:
run.print_statuses()
run.print_output('MyFirstJob')
print(run.get_status('MyFirstJob'))

#### 9. Here's another way to create a workflow.

In [ ]:
# let's clear the workflow
workflow.clear_all()

workflow.chain(
    [
        JobCommand('MyFirstJob', 'echo "First!"'),
        JobCommand('MySecondJob', 'echo "Second!"'),
        JobCommand('MyThirdJob', 'echo "Third!"', run_as_dummy=True),
    ],
    inpath='ChainedJobs'
)

get_graph(workflow)